In [1]:
from peak_resolver.linear_combination_fitting import *
from peak_resolver.get_models_bases import get_models
from peak_resolver.process_chromeleon import process_chromeleon_file
import pickle
import pathlib
from pathlib import Path
import xarray as xr
from xarray import apply_ufunc

In [2]:
def get_basis(root, conc, tmin, tmax, tgrid):
    p = Path(root).resolve()

    files = p.rglob('*.nc')

    ds = []
    for file in list(files):
        print(file)
        data = xr.load_dataset(str(file))
        ds.append(data)
        
    ds = xr.merge(ds)
    # if conc=='0.5':
    #     search = '*6_2*.txt'
    # elif conc=='0.4':
    #     search = '*5_2*.txt'
    # elif conc == '0.3':
    #     search = '*4_2*.txt'
    # elif conc == '0.2':
    #     search = '*3_2*.txt'
    # elif conc == '0.1':
    #     search = '*2_2*.txt'
    # elif conc == '0.05':
    #     search = '*1_2*.txt'
    # files = p.rglob(search)

    # # print(*list(files))


    # for file in list(files):
    #     print(file)
    #     data = process_chromeleon_file(file, tmin, tmax, tgrid = tgrid)
    #     print(data.name)
    return ds
    # return data

In [3]:
basis_root = '/Users/ianbillinge/Documents/yiplab/projects/org_acids_iems/Fanny Raw IC Data/CALIBRATION CURVES/'
ds = get_basis(basis_root, 0.5, tmin = 6.5, tmax = 9, tgrid = np.linspace(0.15, 19.85, 5997))

/Users/ianbillinge/Documents/yiplab/projects/org_acids_iems/Fanny Raw IC Data/CALIBRATION CURVES/ACETIC ACID/Acetic.nc
/Users/ianbillinge/Documents/yiplab/projects/org_acids_iems/Fanny Raw IC Data/CALIBRATION CURVES/BUTYRIC ACID/Butyric.nc
/Users/ianbillinge/Documents/yiplab/projects/org_acids_iems/Fanny Raw IC Data/CALIBRATION CURVES/PROPANOIC ACID/Propanoic.nc
/Users/ianbillinge/Documents/yiplab/projects/org_acids_iems/Fanny Raw IC Data/CALIBRATION CURVES/VALERIC ACID/Valeric.nc
/Users/ianbillinge/Documents/yiplab/projects/org_acids_iems/Fanny Raw IC Data/CALIBRATION CURVES/CAPROIC ACID/Caproic.nc


In [4]:
ds

<xarray.Dataset>
Dimensions:    (t: 9757, Acetic: 6, Butyric: 6, Propanoic: 6, Valeric: 6,
                Caproic: 6)
Coordinates:
  * t          (t) float64 6.501 6.501 6.501 6.501 ... 8.999 8.999 8.999 9.0
  * Acetic     (Acetic) float64 0.05 0.1 0.2 0.3 0.4 0.5
  * Butyric    (Butyric) float64 0.05 0.1 0.2 0.3 0.4 0.5
  * Propanoic  (Propanoic) float64 0.05 0.1 0.2 0.3 0.4 0.5
  * Valeric    (Valeric) float64 0.05 0.1 0.2 0.3 0.4 0.5
  * Caproic    (Caproic) float64 0.05 0.1 0.2 0.3 0.4 0.5
Data variables:
    s          (t, Acetic, Butyric, Propanoic, Valeric, Caproic) float64 -0.0...
    ds         (t, Acetic, Butyric, Propanoic, Valeric, Caproic) float64 1.5e...
    d2s        (t, Acetic, Butyric, Propanoic, Valeric, Caproic) float64 -3.6...

In [5]:
ds

<xarray.Dataset>
Dimensions:    (t: 9757, Acetic: 6, Butyric: 6, Propanoic: 6, Valeric: 6,
                Caproic: 6)
Coordinates:
  * t          (t) float64 6.501 6.501 6.501 6.501 ... 8.999 8.999 8.999 9.0
  * Acetic     (Acetic) float64 0.05 0.1 0.2 0.3 0.4 0.5
  * Butyric    (Butyric) float64 0.05 0.1 0.2 0.3 0.4 0.5
  * Propanoic  (Propanoic) float64 0.05 0.1 0.2 0.3 0.4 0.5
  * Valeric    (Valeric) float64 0.05 0.1 0.2 0.3 0.4 0.5
  * Caproic    (Caproic) float64 0.05 0.1 0.2 0.3 0.4 0.5
Data variables:
    s          (t, Acetic, Butyric, Propanoic, Valeric, Caproic) float64 -0.0...
    ds         (t, Acetic, Butyric, Propanoic, Valeric, Caproic) float64 1.5e...
    d2s        (t, Acetic, Butyric, Propanoic, Valeric, Caproic) float64 -3.6...

In [6]:
def mean(data, dim="t"):

    return apply_ufunc(

        np.abs,

        data,

        input_core_dims=[dim],

        vectorize=True,

    )


    

In [7]:
mean = mean(ds['s'])

ValueError: setting an array element with a sequence.

In [ ]:
mean

In [ ]:
df = ds['ds'].to_dataframe().dropna()

In [ ]:
df

In [ ]:
for side in ds.coords['side'].values:
    for run in ds.coords['run']:
        for exp_time in ds.coords['exp_time']:
            for ace in ds.coords['Acetic']:
                for pro in ds.coords['Propionic']:
                    for but in ds.coords['Butyric']:
                        for val in ds.coords['Valeric']:
                            for cap in ds.coords['Caproic']:
                                print(ds.sel(side=side, 
                                             run=run, 
                                             exp_time=exp_time, 
                                             Acetic=ac,
                                             Propionic=pro,
                                             Butyric=but,
                                             Valeric=val,
                                             Caproic=cap
                                            ).dropna(dim='t', how='all'))
                


In [ ]:
root = '/Users/ianbillinge/Documents/yiplab/projects/org_acids_iems/Fanny Raw IC Data/Calibration Curves'
acids = ['acetic', 'propanoic', 'butyric', 'valeric', 'caproic']
models = get_models(root, acids)
acids = [a.capitalize() for a in acids]
for i in list(models):
    models[i + '_err'] = np.sqrt(np.sum(models[i].resid)**2/models[i].df_resid)

In [ ]:
models['ace'].resid

In [ ]:
models

In [ ]:
a = xr.DataArray([[1,2,3], [4,5,6]], dims=['x', 'y'])
b = xr.DataArray([[-1, -2, -3], [4,5,6]], dims=['x', 'y'])

In [ ]:
b

In [ ]:
def magnitude(a, b):

    # func = lambda x, y: np.sqrt(x**2 + y**2)

    return xr.apply_ufunc(norm, a, b)

def norm(x, y):
    return np.sqrt(x**2 + y**2)

def _inner(x, y):

    result = np.matmul(x[..., np.newaxis, :], y[..., :, np.newaxis])

    return result[..., 0, 0]


def inner_product(a, b, dim):

    return apply_ufunc(_inner, a, b, input_core_dims=[[dim], [dim]])

def v_lin_com(vectors, dim='t', initial_guess=None, bounds = None, method='Nelder-Mead',):

    return apply_ufunc(lin_combination_fitting, target, vectors, initial_guess=initial_guess, bounds = None, method='Nelder-Mead', input_core_dims=[[dim], [dim]])


In [ ]:
v_lin_com()

In [ ]:
a

In [ ]:
b

In [ ]:
inner_product(a,b,'y')